<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">
           <a href="https://bokeh.org/">
           <img
               src="assets/bokeh-transparent.png"
               style="width:50px"
           >
           </a>
       </td>
       <td style="border:none">
           <h1>Bokeh Tutorial</h1>
       </td>
   </tr>
</table>

<div style="float:right;"><h2>08 Plot tools and tooltips</h2></div>

In [1]:
# activate notebook output
from bokeh.io import output_notebook

output_notebook()

# load tutorial data set
import sys

sys.path.append("../data")
from carriers_data import CarrierDataSet

data = CarrierDataSet()

Loading BokehJS ...

This chapter provides an overview of the tools that are available in the Bokeh toolbar.

The Bokeh toolbar is a collection of tools that are displayed together with a plot.
You have seen the toolbar in all previous examples. This chapter is about configuring
the toolbar and its tools.

### General toolbar configuration

By default, the Bokeh toolbar is displayed to the right of every plot. 

You can change this by setting the `toolbar_location` property of the figure:

In [5]:
from bokeh.plotting import figure, show

# 🔁 Try different values for the toolbar_location parameter
p = figure(
    height=300,
    toolbar_location="below",  # can be "above", "below" "left", "right", or None
)

p.line(x=list(range(10)), y=list(range(10)))
show(p)

To completely disable the toolbar for a plot, set `toolbar_location` to `None`.

### Adding and removing tools

When you configure the toolbar with the figure() method, you can specify which tools
to include.

Before you can add a tool, you need to import it from the `bokeh.models` module. You can
then pass a list of tool objects to the `tools` argument:

In [8]:
from bokeh.models import BoxZoomTool, ResetTool  # import the tool objects
from bokeh.plotting import figure

p = figure(
    height=300,
    tools=[BoxZoomTool(), ResetTool()],  # add the tools to the figure
)

p.line(x=list(range(10)), y=list(range(10)))
show(p)

### HoverTool and tooltips

The `HoverTool` is a special tool that displays a tooltip when the user hovers the mouse
over a data point or when you tap on a data point.

To enable tooltips, you need to add a `HoverTool` to the list of tools. You can then
use the `tooltips` argument to specify which data fields to display in the tooltip.

Run the cell below and hover over the data points:

In [19]:
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource

source = ColumnDataSource(
    data={
        "x": [1, 2, 3, 4, 5],
        "y": [3, 7, 8, 5, 1],
    }
)

p = figure(
    toolbar_location=None,
    tools=[HoverTool()],  # add the HoverTool to the figure
    tooltips="Data point @x has the value @y",  # define a tooltip using data from the x and y columns
    height=300,
)

# add renderers
p.circle("x", "y", size=10, source=source)
p.line("x", "y", line_width=2, source=source)

# show the results
show(p)

The text that is displayed in the tooltip uses special syntax to refer to data from the
ColumnDataSource.

In the example above, the tooltip text is `"Data point @x has the value @y"`. The `@x`
and `@y` are replaced with the actual value of the `x` and `y` columns in the
ColumnDataSource at that point.

#### Tooltip fields

There are several **other field names** that you can use in tooltips. See
[Basic tooltips](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html#basic-tooltips)
in the user guide for an overview.

#### Tooltip formatting

Similar to how you defined he number format for axes, you can **format the numbers in
the tooltips**. To define the format of a data field in a tooltip string, add a format
string in curly braces after the field name. For example:
* `"@x{$0.00}"` will display the value of the `x` column with two decimal places.
* `"@y{($ 0.00 a)}"` will display the value of the `y` column as a Dollar value with two
  decimal places and use SI prefixes (k, M, G, etc.) for large numbers.
* `"@x{(0,0)}"` will display the value of the `x` column as a number with a tousand
  separator.
See [Formatting tooltip fields](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html#formatting-tooltip-fields) in the user guide for more examples.

### Multiline tooltips

Tooltips can also have **multiple lines and use multiple data fields**. Use a list of
tuples to specify tooltips with multiple lines.

For example:

```python
hover = HoverTool(
    tooltips=[
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
    ]
)
```

### HoverTool and tooltip example

With the HoverTool, the ColumnDataSource, and the information about customizing visual
properties, you can recreate the "Top 10 carriers by passengers" plot from the demo
dashboard.

First, take a look at the data this plot is visualizing:

In [39]:
# load data from demo data set
df = data.get_biggest_airlines_by_passengers()
df.head(10)

,unique_carrier_name,unique_carrier,passengers,position
0,Southwest Airlines Co.,WN,120715882,1
1,American Airlines Inc.,AA,99225144,2
2,Delta Air Lines Inc.,DL,92633601,3
3,United Air Lines Inc.,UA,58899326,4
4,SkyWest Airlines Inc.,OO,35069897,5
5,Spirit Air Lines,NK,27609122,6
6,JetBlue Airways,B6,22944344,7
7,Alaska Airlines Inc.,AS,21702741,8
8,Frontier Airlines Inc.,F9,19626363,9
9,Republic Airline,YX,17495428,10


This table contains the following columns:
* `unique_carrier_name`: full names of the carriers
* `unique_carrier`: IATA codes of the carriers
* `passengers`: the total number of passengers flown in 2021 for each
of the carriers.
* `position`: the position of the carrier in a ranking based on the number of passengers
  flown in 2021.

The next code cell contains the code to create the plot. After running this cell, you
can hover over the blue bars to see the tooltip for each of the carriers:

In [44]:
from bokeh.models import NumeralTickFormatter, TapTool

# load data for ColumnDataSource from demo data set
source = ColumnDataSource(data.get_biggest_airlines_by_passengers())

# Define tooltips as a list of tuples
TOOLTIPS = [
    ("Position", "@position"),
    ("Carrier", "@unique_carrier_name"),
    ("Passengers", "@passengers{(0,0)}"),
]

# set up a figure with the tooltips to use the TOOLTIPS list
largest_carriers_plot = figure(
    x_range=source.data["unique_carrier_name"][:10],
    title="Top 10 carriers by passengers (domestic routes)",
    height=300,
    tooltips=TOOLTIPS,
)

# create a vbar renderer with the data from the ColumnDataSource
carriers_vbar = largest_carriers_plot.vbar(
    x="unique_carrier_name",
    top="passengers",
    source=source,
    legend_label="Passengers",
    width=0.6,
)

# customize the appearance of the grid and x axis
largest_carriers_plot.xgrid.grid_line_color = None
largest_carriers_plot.yaxis.formatter = NumeralTickFormatter(format="0,0")
largest_carriers_plot.xaxis.major_label_orientation = 0.8  # rotate labels by roughly pi/4

show(largest_carriers_plot)

# Next section

[TBD, placeholder]